# IMPORTS

In [1]:
import numpy as np
import random

import dimod
import dwave
import dwave.system

# Bin Packing

## Problem Formulation

In [5]:
#here i generate a synthetic(random) set of items

num_items=15
item_weight_range = [3,7]
weights = np.random.randint(*item_weight_range, num_items)
bin_capacity = int(5 * np.mean(weights))
print("Problem: pack a total weight of {} into bins of capacity {}.".format(
      sum(weights), bin_capacity)) 

Problem: pack a total weight of 63 into bins of capacity 21.


In [18]:
#instantiate the cqm
cqm = dimod.ConstrainedQuadraticModel()

## Objective Function

### Objective Function

In [7]:
bin_used = [dimod.Binary(f'bin_used_{j}') for j in range(num_items)]

In [64]:
cqm.set_objective(sum(bin_used))

### Contraints

**Each item can go into only one bin.**

Define new binary variables $ x_{ij} = 1 $ if $item_i \in bin_j$, 0 otherwise.

The constraint can then be expressed as:
$$\sum_j x_{ij} == 1 \quad \forall i \in [1,nitems]$$

In [19]:
item_in_bin = [ [dimod.Binary(f'item_{i}_in_bin_{j}') for j in range(num_items)] for i in range(num_items)]

for i in range(num_items):
    one_bin_per_item = cqm.add_constraint( sum(item_in_bin[i]) == 1, label=f'item_placing_{i}')

**Each bin has a limited capacity $\bold c$**

$$\sum_i x_{ij} \cdot \omega_i \leq c$$

In [25]:
for j in range(num_items):
    bin_up_to_capacity = cqm.add_constraint(
        sum(weights[i] * item_in_bin[i][j] for i in range(num_items)) - bin_used[j] * bin_capacity <= 0,
        label=f'capacity_bin_{j}'
    )

In [27]:
len(cqm.variables)

240

## Solve the Problem (Hybrid)

In [65]:
sampler = dwave.system.LeapHybridCQMSampler()

In [66]:
sampleset = sampler.sample_cqm(cqm,
                               time_limit=180,
                               label="SDK Examples - Bin Packing")  
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)  
if len(feasible_sampleset):      
   best = feasible_sampleset.first
   print("{} feasible solutions of {}.".format(
      len(feasible_sampleset), len(sampleset)))

141 feasible solutions of 145.


In [67]:
best.sample.items()

dict_items([('bin_used_0', 0.0), ('bin_used_1', 0.0), ('bin_used_10', 0.0), ('bin_used_11', 0.0), ('bin_used_12', 0.0), ('bin_used_13', 0.0), ('bin_used_14', 1.0), ('bin_used_2', 1.0), ('bin_used_3', 0.0), ('bin_used_4', 0.0), ('bin_used_5', 0.0), ('bin_used_6', 1.0), ('bin_used_7', 0.0), ('bin_used_8', 0.0), ('bin_used_9', 0.0), ('item_0_in_bin_0', 0.0), ('item_0_in_bin_1', 0.0), ('item_0_in_bin_10', 0.0), ('item_0_in_bin_11', 0.0), ('item_0_in_bin_12', 0.0), ('item_0_in_bin_13', 0.0), ('item_0_in_bin_14', 0.0), ('item_0_in_bin_2', 1.0), ('item_0_in_bin_3', 0.0), ('item_0_in_bin_4', 0.0), ('item_0_in_bin_5', 0.0), ('item_0_in_bin_6', 0.0), ('item_0_in_bin_7', 0.0), ('item_0_in_bin_8', 0.0), ('item_0_in_bin_9', 0.0), ('item_10_in_bin_0', 0.0), ('item_10_in_bin_1', 0.0), ('item_10_in_bin_10', 0.0), ('item_10_in_bin_11', 0.0), ('item_10_in_bin_12', 0.0), ('item_10_in_bin_13', 0.0), ('item_10_in_bin_14', 0.0), ('item_10_in_bin_2', 1.0), ('item_10_in_bin_3', 0.0), ('item_10_in_bin_4', 0.0)

In [72]:
selected_bins = [key for key, val in best.sample.items() if 'bin_used' in key and val==1.0]   
print("{} bins are used.".format(len(selected_bins)))  

3 bins are used.


In [69]:
selected_bins

['bin_used_14', 'bin_used_2', 'bin_used_6']

In [34]:
def get_indices(name):
    return [int(digs) for digs in name.split('_') if digs.isdigit()]

In [73]:
for bin in selected_bins:                        
    in_bin = [key for key, val in best.sample.items() if
       "_in_bin" in key and
       get_indices(key)[1] == get_indices(bin)[0]
       and val]
    b = get_indices(in_bin[0])[1]
    w = [weights[get_indices(item)[0]] for item in in_bin]
    print("Bin {} has weights {} for a total of {}.".format(b, w, sum(w)))

Bin 14 has weights [3, 3, 4, 6, 5] for a total of 21.
Bin 2 has weights [4, 6, 3, 5, 3] for a total of 21.
Bin 6 has weights [5, 5, 3, 3, 5] for a total of 21.


# Constrained Scheduling

## Problem Formulation

4 binary variables to formulate the problem: t(time of day), v(venue), l(lenght), p(participation)

## Constraints

Constraint 1: During business hours, all meetings must be attended in person at the office.

Constraint 2: During business hours, participation in meetings is mandatory.

Constraint 3: Outside business hours, meetings must be teleconferenced.

Constraint 4: Outside business hours, meetings must not exceed 30 minutes.

## Constraints as Penalties

1. If t == 1 then v == 1, so the penalty is t - tv
2. if t == 1 then p == 1, $\qquad\qquad\qquad$ t - tp
3. if t == 0 then v == 0, $\qquad\qquad\qquad$ v - vt
4. if t == 0 then l == 1, $\qquad\qquad\qquad$ 1 + tl - t - l

In [2]:
bqm = dimod.BinaryQuadraticModel({'t':1, 'v':1, 'l':-1},  #linear terms
                                 {'tv':-2, 'tl':1, 'tp':-1},  #quadratic terms
                                 1,  #constant
                                 'BINARY')  #dtype

## Solve the problem (CPU)

In [4]:
sampler = dimod.reference.samplers.ExactSolver()
sampleset = sampler.sample(bqm)

In [7]:
print(sampleset.lowest(atol=.5))

   l  p  t  v energy num_oc.
0  1  0  0  0    0.0       1
1  1  1  0  0    0.0       1
2  1  1  1  1    0.0       1
3  0  1  1  1    0.0       1
['BINARY', 4 rows, 4 samples, 4 variables]


## Solve the Problem (QPU)

In [3]:
sampler = dwave.system.EmbeddingComposite(dwave.system.DWaveSampler())

In [ ]:
sampleset = sampler.sample(bqm, num_reads=5000, label='SDK Examples - Scheduling')

In [ ]:
print(sampleset.lowest(atol=.5))

# Job-Shop Scheduling

## Instance generation and Utils

In [16]:
def generate_random_instance(n_machine, n_job, max_duration):

    instance = {}
    machine_list = list(range(n_machine))

    for i in range(n_job):
        random.shuffle(machine_list)
        instance[i] = []
        for j in range(n_machine):
            instance[i].append((machine_list[j], random.randint(0,max_duration)))
    
    return instance

'job_n: machine_n, duration'


'job_n: machine_n, duration'

In [37]:
class JSSCQM:

    def __init__(self, instance):

        self.instance = instance

        self.num_jobs = len(instance)
        self.num_machines = len(instance[0])
        
        self.max_makespan = 0
        for k in range(self.num_jobs):
            for i in range(self.num_machines):
                self.max_makespan += instance[k][i][1]

        #start time on machine i of job j
        self.x = { (i,j): dimod.Integer('x{}_{}'.format(i,j), lower_bound=0, upper_bound=self.max_makespan) 
                for j in range(self.num_jobs) for i in range(self.num_machines)}
        
        #equals 1 if job j precedes job k on machine i
        self.y = {(i,j,k): dimod.Binary('y{}_{}_{}'.format(i,j,k)) 
                  for i in range(self.num_jobs) 
                  for j in range(self.num_jobs)  
                  for k in range(self.num_machines)}
        
        self.makespan = dimod.Integer('makespan', lower_bound=0, upper_bound=self.max_makespan)

        self.cqm = dimod.ConstrainedQuadraticModel()

        self.define_objective_function()
        self.add_overlap_constraint()
        self.add_precedence_constraint()

    def define_objective_function(self):

        self.cqm.set_objective(self.makespan)

        for i in range(self.num_jobs):
            last_machine = self.instance[i][-1][0]

            self.cqm.add_constraint(
                self.makespan -self.x[(i, last_machine)] >= self.instance[i][-1][1]
            )

    def add_precedence_constraint(self):

        for j in range(self.num_jobs):
            for t in range(self.num_machines):
                m_curr = self.instance[j][t][0]
                m_prev = self.instance[j][t-1][0]

                self.cqm.add_constraint(
                    self.x[(j,m_curr)] - self.x[(j,m_prev)] >= instance[j][t-1][1]
                )

    def add_overlap_constraint(self):
        for j in range(self.num_jobs):
            for k in range(self.num_jobs):
                if j<k:
                    for i in range(self.num_machines):
                        task_k = self.get_machine_task(k,i)
                        task_j = self.get_machine_task(j,i)

                        if self.instance[k][task_k][1] > 0 and self.instance[j][task_j][1] > 0:

                            self.cqm.add_constraint(
                                self.x[(j,i)] - self.x[(k,i)] +

                                (self.instance[k][task_k][1] -
                                 self.instance[j][task_j][1])*self.y[(j,k,i)] +

                                 2*self.y[(j,k,i)]*(self.x[(k,i)] - self.x[(j,i)]) >=
                                 self.instance[k][task_k][1],

                                 label = 'OneJobj{}_j{}_m{}'.format(j,k,i)
                            )

    def get_machine_task(self, j, machine):
        count = 0
        for t in range(self.num_machines):
            if self.instance[j][t][0] == machine:
                return count
            else:
                count +=1


    
        

    


        

    


SyntaxError: incomplete input (3739232535.py, line 94)

In [38]:
instance = generate_random_instance(5,5,10)

jss = JSSCQM(instance)

In [36]:
instance[0][0,0]

TypeError: list indices must be integers or slices, not tuple

In [26]:
jss.cqm.